In [207]:
import pandas as pd
from pulp import *
import openpyxl
import re
import plotly.express as px

In [208]:
wb = openpyxl.Workbook()
ws = wb.active

In [209]:
players = pd.read_excel("auc_values_ALL.xlsx",
                      usecols= ["Value", "Pts", "Name", "Pos"])

In [210]:
players.head()

,Value,Name,Pos,Pts
0,72,Jonathan Taylor,RB,287
1,63,Derrick Henry,RB,266
2,55,Dalvin Cook,RB,243
3,54,Nick Chubb,RB,239
4,54,Christian McCaffrey,RB,239


In [211]:
fig = px.scatter(players, x="Value", y="Pts", color=None, hover_name="Name")
fig

In [212]:
availables = players.groupby(["Value", "Pts", "Name", "Pos"]).agg('count')

In [213]:
availables = availables.reset_index()

In [214]:
salaries = {}
points = {}
teams = {},

In [215]:
for Pos in availables.Pos.unique():
    available_pos = availables[availables.Pos == Pos]
    salary = list(available_pos[['Name', 'Value']].set_index("Name").to_dict().values())[0]
    point = list(available_pos[['Name', 'Pts']].set_index("Name").to_dict().values())[0]

    salaries[Pos] = salary
    points[Pos] = point

In [216]:
pos_num_available = {
    "QB": 1,
    "RB": 3,
    "WR": 3,
    "TE": 1,
    "K": 1,
    "Def": 1
                    }

In [217]:
salary_cap = 190

In [218]:
#create lineup loop
for lineup in range(1,166):
    _vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}

In [219]:
for lineup in range(1,166):
    _vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}

    prob = LpProblem("Fantasy", LpMaximize)

In [220]:
for lineup in range(1,166):
    _vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}

    prob = LpProblem("Fantasy", LpMaximize)

    rewards = []
    costs = []
    position_constraints = []

In [221]:
for lineup in range(1,166):
    _vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}

    prob = LpProblem("Fantasy", LpMaximize)

    rewards = []
    costs = []
    position_constraints = []

    for k, v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        prob += lpSum([_vars[k][i] for i in v]) == pos_num_available[k]

In [222]:
#add the costs(Salary) and rewards(Points) into the optimization calculation
for lineup in range(1,166):
    _vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}

    prob = LpProblem("Fantasy", LpMaximize)
    rewards = []
    costs = []
    position_constraints = []
    total_score = []

    for k, v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        prob += lpSum([_vars[k][i] for i in v]) == pos_num_available[k]

    prob += lpSum(rewards)
    prob += lpSum(costs) <= salary_cap
    if not lineup == 1:
        prob += (lpSum(rewards) <= -0.01)
    prob.solve()

In [223]:
for lineup in range(1,166):
    _vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}

    prob = LpProblem("Fantasy", LpMaximize)
    rewards = []
    costs = []
    position_constraints = []
    total_score = []
    for k, v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        prob += lpSum([_vars[k][i] for i in v]) == pos_num_available[k]

    prob += lpSum(rewards)
    prob += lpSum(costs) <= salary_cap
    if not lineup == 1:
        prob += (lpSum(rewards) <= total_score-0.01)
    prob.solve()

    score= str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    colnum = 1

    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        if v.varValue !=0:
            ws.cell(row=lineup, column=colnum).value = v.name
            colnum +=1
    total_score = eval(score)
    ws.cell(row=lineup, column=colnum).value = total_score
    print(lineup, total_score)

1 1659.0


TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [ ]:
#wb.save(r"\playerLists\190_3rb.xlsx")

In [ ]:
#wb.save(r"\playerLists\halfsal.xlsx")

In [142]:
#references
# https://www.nicksniche.net/lineup-optimization/lineup-optimizer-part-1